In [1]:
import os


In [4]:
os.chdir('../')

In [5]:

!pwd

/Users/sumi/Desktop/DS-PROJECT/end-to-end-DS-project


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from src.DSproject.constants import *
from src.DSproject.utils.common import read_yaml, create_directories


In [25]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->DataIngestionconfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [22]:
import os
import urllib.request as request
from src.DSproject import logger
import zipfile

In [28]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config=config
      
    # Downloading the zip file    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists")    
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-09 20:14:53,055: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-09 20:14:53,057: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-09 20:14:53,058: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-09 20:14:53,059: INFO: common: created directory at: artifacts]
[2024-11-09 20:14:53,060: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-09 20:14:53,060: INFO: 2572878169: file already exists]
